In [5]:
#r "nuget:DotNet.Interactive.Extensions.Revit"
#r "nuget:Revit.RevitApi.x64, 2023.0.0"
#r "nuget:Revit.RevitApiUi.x64, 2023.0.0"
#i "C:\git\bim-net-interactive\src\DotNet.Interactive.Extensions.Revit\bin\Debug\"
//using DotNet.Interactive.Extensions.Revit;
using Autodesk.Revit.DB;
using Autodesk.Revit.UI;

Restore sources C:\git\bim-net-interactive\src\DotNet.Interactive.Extensions.Revit\bin\Debug\ Installed Packages DotNet.Interactive.Extensions.Revit, 1.51.0 Revit.RevitApi.x64, 2023.0.0 Revit.RevitApiUi.x64, 2023.0.0

In [2]:

//Use the magic command #!connect to add a revit kernel.
#!connect revit --kernel-name revit24 --revit-version 2014

Kernel added: #!revit24

In [21]:


string filePath = @"";
string fileName = @"File.ifc";

var fileInfo = new { fileName = fileName, filePath = filePath};
display(fileInfo)

fileName,File.ifc
filePath,C:\tmp\


In [8]:
//#!revit24

//Because the revit kernel is run out of process embedded in the revit application we need to use the set magic command
#!set --value @csharp:fileInfo --name fileInfo

 UIDocument uidoc = default;

 Document doc = uidoc.Document;
 // Set export options
    IFCExportOptions ifcExportOptions = new IFCExportOptions();
    
    // Optionally, you can customize export options further, e.g., set specific configurations.
    // ifcExportOptions.SpaceBoundaries = IFCExportSpaceBoundaries.Boundaries;
    // ifcExportOptions.SplitWallsAndColumns = true;
    // ...

    // Perform the export
   Transaction transaction = new Transaction(doc, "Export IFC");
   transaction.Start("start");

        // Use the Export method of the Document class
    doc.Export(fileInfo.fileName, fileInfo.filePath, ifcExportOptions);

    transaction.Commit();

    
    

Error: System.IO.FileNotFoundException: Could not load file or assembly 'C:\Users\Joel\.nuget\packages\revit.revitapiui.x64\2023.0.0\lib\net48\RevitAPIUI.dll'. Det går inte att hitta den angivna modulen.
File name: 'C:\Users\Joel\.nuget\packages\revit.revitapiui.x64\2023.0.0\lib\net48\RevitAPIUI.dll'
   at Submission#9.<<Initialize>>d__0.MoveNext()
   at System.Runtime.CompilerServices.AsyncMethodBuilderCore.Start[TStateMachine](TStateMachine& stateMachine)
   at Submission#9.<Initialize>()
   at Submission#9.<Factory>(Object[] submissionArray)
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Now we want to analyze to exported IFC and list all levels using a open source library. In this case we use XBim but you could also use IFCOpenShell with Python.

In [22]:
#r "nuget:Xbim.Essentials"
using Xbim.Ifc;
using Xbim.Ifc4.GeometricModelResource;
using Xbim.Ifc4.Kernel;
using Xbim.Ifc4.ProductExtension;
using System.IO;

// Replace 'path/to/your/ifcfile.ifc' with the actual path to your IFC file
string ifcFilePath = Path.Combine(fileInfo.filePath, fileInfo.fileName);
// Open the IFC file
display("test");
using (var model = IfcStore.Open(ifcFilePath))
{
    if (model != null)
    {
        var levels = model.Instances.OfType<Xbim.Ifc2x3.Interfaces.IIfcBuildingStorey>();

            foreach (var level in levels)
            {
                display($"Level Name: {level.Name}");
            }
    }
    else
    {
        display("Failed to open the IFC file.");
    }
}

Restore sources C:\git\bim-net-interactive\src\DotNet.Interactive.Extensions.Revit\bin\Debug\ Installed Packages Xbim.Essentials, 5.1.341

test

Error: System.IO.FileNotFoundException: C:\tmp\File.ifc file was not found
   at Xbim.Ifc.IfcStore.Open(String path, XbimEditorCredentials editorDetails, Nullable`1 ifcDatabaseSizeThreshHold, ReportProgressDelegate progDelegate, XbimDBAccess accessMode, Int32 codePageOverride) in D:\a\1\s\Xbim.Ifc\IfcStore.cs:line 249
   at Submission#23.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)